In [1]:
import plotly.express as px
import pandas as pd


In [36]:
# data as stacked bar charts
data = {
    "Organism": [],
    "Read Count": [],
    "Sample": []
}
samples = []
min_reads = 0
total_others = []

with open('kraken2.csv', 'r') as f:
    raw_data = f.read().split("\n")
    
    samples = ["Sample "+x for x in raw_data[0].split(",") if x != ""]
    for s in samples:
        total_others.append(0)
    
    for r in raw_data[1:]:
        do_add = False
        tmp_new = [0 for s in samples]
        
        tmp_r = r.split(",")
        if len(tmp_r) > 1: # dealing with the last row
            org = "|".join([x for x in tmp_r[:-len(samples)] if x != "-"])

            for i in range(0, len(samples)):
                tmp_new[i] = tmp_r[-len(samples)+i]
                
                if tmp_new[i] != "NaN" and int(tmp_new[i]) >= min_reads:
                    do_add = True
                
            if do_add:
                for i in range(0, len(samples)):
                    data["Organism"].append(org)
                    data["Read Count"].append(tmp_new[i])
                    data["Sample"].append(samples[i])
            else:
                for i in range(0, len(samples)):
                    if tmp_new[i] != "NaN":
                        total_others[i] += int(tmp_new[i])
    
for i in range(0, len(samples)):
    data["Organism"].append("other")
    data["Read Count"].append(total_others[i])
    data["Sample"].append(samples[i])
    
df = pd.DataFrame.from_dict(data)
df["Read Count"] = df["Read Count"].astype(float)


In [37]:
fig = px.bar(df, x="Sample", y="Read Count", color='Organism')
fig.update_layout(showlegend=False)
fig.write_html("kraken2.html")
fig.show()


In [104]:
# data as stacked bar charts
data = {
    "Function": [],
    "Abundance RPKs": [],
    "Sample": []
}
samples = []
min_abundance = 10000 # reads per kilobase

with open('genefamilies.csv', 'r') as f:
    raw_data = f.read().split("\n")
    
    samples = ["Sample "+x for x in raw_data[0].split(",") if x != ""]
    samples = samples[:-1] # skip out the function bit

    # this is a bit redundant, but is fast enough for me not to care
    all_functions = {}
    
    for r in raw_data[1:]:
        tag = r.split(",")[0]
        tmp_f = r.split(",")[-1].split("DR   GO;")

        for func in tmp_f[1:]: # skipping the initial empty split
            if func not in all_functions:
                all_functions[func] = []
                
            all_functions[func].append(tag)

    all_data = {s: {} for s in samples} 
    
    for r in raw_data[1:]:
        tmp_r = r.split(",")
        
        if len(tmp_r) > 1: # dealing with the last row
            for i in range(0, len(samples)): # ignoring the last one because it's the function column
                all_data[samples[i]][tmp_r[0]] = float(tmp_r[(i+1)]) if tmp_r[(i+1)] != "NaN" else 0.0

    sample_functions = {s: {} for s in samples}
    
    for sample in samples:
        for func, genes in all_functions.items():
            abundance = sum([all_data[sample][g] for g in genes if g in all_data[sample]])
            
            sample_functions[sample][func] = abundance
            
            if abundance >= min_abundance:
                data["Function"].append(func.strip())
                data["Abundance RPKs"].append(abundance)
                data["Sample"].append(sample)
        
df = pd.DataFrame.from_dict(data)
df["Abundance RPKs"] = df["Abundance RPKs"].astype(float)


In [106]:
fig = px.bar(df, x="Sample", y="Abundance RPKs", color='Function')
fig.update_layout(showlegend=False)
fig.write_html("humann2.html")
fig.show()
